Natural Language Processing: Pretraining Word Embeddings
This script covers the implementation of concepts of word embeddings, specifically focusing on:
1.  **Word2Vec Pretraining**: Implementing the Skip-Gram model with Negative Sampling from scratch.
2.  **Subword Embeddings**: Implementing Byte Pair Encoding (BPE) to handle subword tokenization.
3.  **Word Similarity and Analogy**: Using pretrained GloVe vectors to solve semantic analogy tasks.

**Note:** All functions are defined from scratch using `torch` and standard Python libraries i.e. no external framework has been used.

Below code block establishes the **infrastructure layer** for the entire NLP project. Before implementing complex models like Word2Vec or BERT, we need reliable tools to handle data acquisition, hardware acceleration, and performance monitoring.

#### **1. Purpose and Function**

The code serves three critical functions necessary for any Deep Learning workflow:

1.  **Data Acquisition:** It automates the fetching of specific datasets (Penn Tree Bank for training, GloVe for analogies) from remote servers. This ensures the code is reproducible on any machine.
2.  **Hardware Abstraction:** It abstracts the check for GPU availability (`try_gpu`). Deep learning models rely on matrix multiplications that run significantly faster on GPUs (CUDA) than CPUs.
3.  **Training Telemetry:** It provides tools (`Timer`, `Accumulator`) to measure how fast the model trains (tokens/sec) and to track the loss over time.

-----

#### **2. Detailed Theoretical Breakdown**

**A. Dataset Downloading (`download_url`, `download_extract`)**
In NLP, datasets are often large compressed archives.

  * **Streaming Downloads:** The code uses `requests.get(..., stream=True)`. This is crucial because datasets can be larger than available RAM. Streaming allows the file to be written to disk in chunks rather than loaded entirely into memory first.
  * **Presets:** The `DATA_URLS` dictionary maps shorthand names ('ptb', 'glove') to specific AWS S3 URLs. This simplifies the user experience in later cells—instead of typing a long URL, you just request `'ptb'`.

**B. Hardware Acceleration (`try_gpu`)**

  * **Tensor Operations:** Neural networks involve massive matrix multiplications. On a CPU, these are done sequentially or with limited parallelism. GPUs have thousands of cores designed for parallel arithmetic.
  * **Device Agnosticism:** This function allows the code to run anywhere. If a GPU is detected, it returns `cuda:0`; otherwise, it falls back to `cpu`. This prevents the code from crashing on non-GPU machines (like standard laptops).

**C. Metric Tracking (`Accumulator`)**

  * **The Stochastic Gradient Descent (SGD) Problem:** In SGD, we update weights after every small "batch" of data. To understand how the model is doing overall, we need to calculate the average loss over an entire "epoch" (one full pass through the data).
  * **Accumulation:** The `Accumulator` class maintains a running total of multiple metrics simultaneously (e.g., `[total_loss, total_tokens_processed]`). At the end of an epoch, we calculate `Average Loss = Total Loss / Total Tokens`.

-----

#### **3. Key Code Lines & Their Roles**

**1. Streaming Download Logic**

```python
r = requests.get(url, stream=True)
for chunk in r.iter_content(chunk_size=chunk_size):
    fd.write(chunk)
```

  * **Role:** This ensures efficient memory usage. By writing `chunk_size` bytes at a time, we can download gigabyte-sized datasets (like GloVe) without crashing the program due to memory overflow.

**2. GPU Detection**

```python
if torch.cuda.device_count() >= i + 1:
    return torch.device(f'cuda:{i}')
```

  * **Role:** This interacts with the NVIDIA driver via PyTorch. It checks if the requested GPU index (`i`) physically exists. This line is the gateway to accelerating training speed by 10x-50x.

**3. The Accumulation Logic**

```python
self.data = [a + float(b) for a, b in zip(self.data, args)]
```

  * **Role:** This Python list comprehension performs element-wise addition.
      * If `self.data` is `[100.0, 50]` (current sums) and `args` is `[2.5, 10]` (new batch values), the new state becomes `[102.5, 60]`.
      * This is used inside the training loop to update `sum_loss` and `num_tokens` in a single line of code, keeping the training loop clean.

In [ ]:
#  Utilities and Helper Functions

# We start by defining essential utility functions that are handy and prove 
# to be very useful. These tools are for downloading datasets, timing 
# execution, and tracking training metrics.

import collections
import math
import os
import random
import sys
import time
import zipfile
import requests
import torch
from torch import nn
from torch.utils import data

# Dataset Downloading Utilities 

def download_url(url, save_path, chunk_size=128):
    """Downloads a file from a URL to a local path with a progress stream."""
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

def download_extract(name, folder=None):
    """
    Download and extract a zip file from the D2L data repository.
    Returns the directory path containing the extracted files.
    """
    DATA_URLS = {
        'ptb': 'http://d2l-data.s3-accelerate.amazonaws.com/ptb.zip',
        'glove.6b.50d': 'http://d2l-data.s3-accelerate.amazonaws.com/glove.6B.50d.zip'
    }
    
    if name not in DATA_URLS:
        # Fallback or error handling if needed
        print(f"Warning: {name} not in preset URLs.")
        return name
    
    url = DATA_URLS[name]
    fname = os.path.join('./data', name + ".zip")
    os.makedirs('./data', exist_ok=True)
    
    # Download if not exists
    if not os.path.exists(fname):
        print(f"Downloading {name} from {url}...")
        download_url(url, fname)
    
    # Extract
    base_dir = os.path.dirname(fname)
    data_dir, _ = os.path.splitext(fname)
    if folder:
        data_dir = os.path.join(base_dir, folder)
        
    if not os.path.exists(data_dir):
        print(f"Extracting {fname}...")
        with zipfile.ZipFile(fname, 'r') as zip_ref:
            zip_ref.extractall(base_dir)
            
    return data_dir

# Hardware and Training Utilities

def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

class Timer:
    """Record multiple running times."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """Start the timer."""
        self.tik = time.time()

    def stop(self):
        """Stop the timer and record the time in a list."""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

class Accumulator:
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def __getitem__(self, idx):
        return self.data[idx]

### Dataset Processing and Subsampling

### 1\. Purpose and Function

This code block is the **Data Preprocessing Pipeline**. Its primary goal is to convert human-readable text (strings) into machine-readable data (numerical indices) while optimizing the dataset for efficient training. It performs three critical tasks:

1.  **Loading**: Reads the Penn Tree Bank (PTB) corpus and splits it into tokens (words).
2.  **Indexing (Vocab)**: Creates a dictionary mapping every unique word to a unique integer ID ($0, 1, 2, ...$).
3.  **Subsampling**: Removes extremely frequent words (like "the", "a", "in") to speed up training and focus the model on semantically rich words.

### 2\. Theoretical Explanation

#### A. Vocabulary Construction

Neural networks cannot process strings directly; they require numerical vectors. The `Vocab` class constructs a bijection (two-way mapping) between words and integers.

  * **The Unknown Token (`<unk>`)**: In any real-world corpus, there will be rare words. To keep the model memory-efficient, we often discard words that appear fewer times than `min_freq`. These are mapped to a special token `<unk>` (index 0). This ensures the model can handle unseen words during inference.

#### B. Subsampling (Zipf's Law)

Natural language follows **Zipf's Law**, which states that the frequency of a word is inversely proportional to its rank.

  * The most frequent words (stopwords like "the", "is", "a") appear millions of times but carry very little specific semantic meaning.
  * Context windows like `("the", "apple")` appear frequently but teach the model very little about the meaning of "apple" compared to `("eating", "apple")`.

**Subsampling** addresses this by probabilistically discarding words. The formula used is:
$$P(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right)$$
Where:

  * $f(w_i)$ is the frequency (count/total) of the word $w_i$.
  * $t$ is a threshold hyperparameter (typically $10^{-4}$).

If $f(w_i) > t$ (the word is frequent), the probability of discarding it increases. This balances the dataset, allowing the model to see more rare, information-rich words within the limited training epochs.

### 3\. Key Code Lines

  * **`counter = collections.Counter(...)`**:

      * **Role**: Computes the raw frequency of every word in the corpus. This statistic is the foundation for both Vocabulary building (filtering rare words) and Subsampling (filtering frequent words).

  * **`self.token_to_idx = {token: idx ...}`**:

      * **Role**: Creates the hash map that provides $O(1)$ lookups to convert any word string into its integer ID. This is critical for performance during the data loading phase.

  * **`return (random.uniform(0, 1) < math.sqrt(1e-4 / counter[token] * num_tokens))`**:

      * **Role**:

This single line implements the **Subsampling Formula**.
*  `counter[token] * num_tokens` approximates the relative frequency $f(w_i)$.
*  `1e-4` is the threshold $t$.
*  If the random number is *less* than the calculated threshold term, the function returns `True` (keep the word). If the word is very frequent, the threshold term becomes small, making it harder to satisfy the condition, thus dropping the word.

In [2]:
#  Dataset Processing (PTB)
# We will use the Penn Tree Bank (PTB) dataset for training Word2Vec. 
# This section handles:
# 1.  Reading: Loading the text data.
# 2.  Vocabulary: Building a vocabulary mapping tokens to indices.
# 3.  Subsampling: Removing frequent words (like "the", "a") to speed up training and improve quality.


def read_ptb():
    """Load the PTB dataset into a list of text lines."""
    data_dir = download_extract('ptb')
    # PTB training file location
    with open(os.path.join(data_dir, 'ptb.train.txt')) as f:
        raw_text = f.read()
    return [line.split() for line in raw_text.split('\n')]

class Vocab:
    """Vocabulary for mapping text tokens to numerical indices."""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        # Flatten token list
        counter = collections.Counter([token for line in tokens for token in line])
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        
        # The list of unique tokens
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        return 0

def subsample(sentences, vocab):
    """
    Subsample high-frequency words.
    Words are discarded with probability P(w) = 1 - sqrt(t / f(w)).
    """
    # Exclude unknown tokens from subsampling logic
    sentences = [[token for token in line if vocab[token] != vocab.unk]
                 for line in sentences]
    
    counter = collections.Counter([token for line in sentences for token in line])
    num_tokens = sum(counter.values())

    def keep(token):
        return (random.uniform(0, 1) < math.sqrt(1e-4 / counter[token] * num_tokens))

    return ([[token for token in line if keep(token)] for line in sentences], counter)

Belw code block implements the core data generation logic for the **Skip-Gram** model. While the previous block converted text to integers, this block transforms those integers into the actual **(input, target)** pairs required for supervised learning, specifically applying the **Negative Sampling** optimization strategy.

#### **1. Purpose and Function**

The code serves three main purposes:

1.  **Context Window Generation:** For every word in the text (the "center" word), it identifies which words surround it (the "context" words) to form positive training examples.
2.  **Noise Distribution Calculation:** It calculates a specific probability distribution for sampling "noise" words, raising word frequencies to the power of 0.75.
3.  **Negative Sampling:** For every positive pair (center, context), it generates $K$ "negative" words (noise) that *do not* appear in the context. This prepares the data for the binary classification objective used in approximate training.

-----

#### **2. Detailed Theoretical Breakdown**

**A. Skip-Gram Context Extraction (`get_centers_and_contexts`)**
In the Skip-Gram model, the goal is to predict context words ($w_o$) given a center word ($w_c$).

  * **The Sliding Window:** We slide a window over the text. The word in the middle is the center; words to the left and right are the context.
  * **Dynamic Window Size:** Notice the code uses `random.randint(1, max_window_size)`. In the original Word2Vec paper, the window size isn't fixed. By randomizing the window size for each center word, the model effectively gives **higher weight to closer words** (which are always included) and lower weight to distant words (which are sometimes excluded).

**B. Negative Sampling (NEG) Theory**
Standard Softmax requires calculating the probability for *every* word in the vocabulary (often 50,000+) to normalize the result. This is computationally expensive.

  * **The Solution:** Instead of a massive multi-class classification, we turn it into a **Binary Classification** problem.
      * Is the pair `(word_A, word_B)` real? (Target = 1)
      * Is the pair `(word_A, word_Random)` fake? (Target = 0)
  * **The Dataset:** To train this, we need "Negative Samples"—words that *could* have appeared but didn't.

**C. The 0.75 Power Heuristic (`get_negatives`)**
When selecting noise words, we don't pick them uniformly (randomly) nor strictly by frequency. We use a specific distribution:
$$P(w_i) = \frac{f(w_i)^{0.75}}{\sum_{j} f(w_j)^{0.75}}$$

  * **Why 0.75?** This empirical trick dampens the dominance of extremely frequent words (like "the") and slightly boosts the probability of sampling rare words as negatives. This ensures the model learns to distinguish the center word not just from "the" (which is easy) but from other meaningful words (which is harder and more educational for the embeddings).

-----

#### **3. Key Code Lines & Their Roles**

**1. Dynamic Context Window**

```python
window_size = random.randint(1, max_window_size)
indices = list(range(max(0, i - window_size), min(len(line), i + 1 + window_size)))
```

  * **Role:** This implements the stochastic window size. If `max_window_size` is 5, sometimes the window is 1 (very local context), sometimes 5 (broad context). This implicitly re-weights semantic relationships based on distance.

**2. The 0.75 Power Heuristic**

```python
sampling_weights = [counter[vocab.to_tokens(i)]**0.75 for i in range(1, len(vocab))]
```

  * **Role:** This is the exact implementation of the Word2Vec noise distribution formula. It transforms raw counts into the specific weights used for Negative Sampling.

**3. Efficient Sampling Cache**

```python
self.candidates = random.choices(self.population, self.sampling_weights, k=10000)
```

  * **Role:** Generating random numbers is computationally slow in Python. Instead of calling `random.choices` (which calculates cumulative weights every time) for every single word, this class generates 10,000 choices at once and serves them from a cache. This provides a significant speedup during data loading.

**4. Rejection Sampling**

```python
if neg not in contexts:
    negatives.append(neg)
```

  * **Role:** This ensures data integrity. If we accidentally sample a "noise" word that actually appears in the true context window, we reject it. We must not teach the model that a word is "negative" (label 0) if it is actually present in the context.

In [3]:
#  Context Extraction and Negative Sampling
# To train Skip-Gram, we need to generate pairs of (center, context) words.
# 1.  Context Extraction: For each center word, we select a window of surrounding words.
# 2.  Negative Sampling: For every positive (center, context) pair, we sample K noise words that do not appear in the context.


def get_centers_and_contexts(corpus, max_window_size):
    """Return center words and context words in skip-gram."""
    centers, contexts = [], []
    for line in corpus:
        if len(line) < 2:
            continue
        centers += line
        for i in range(len(line)):
            # Random window size between 1 and max_window_size
            window_size = random.randint(1, max_window_size)
            indices = list(range(max(0, i - window_size),
                                 min(len(line), i + 1 + window_size)))
            indices.remove(i) # Exclude center word itself
            contexts.append([line[idx] for idx in indices])
    return centers, contexts

class RandomGenerator:
    """Randomly draw among {1, ..., n} according to n sampling weights."""
    def __init__(self, sampling_weights):
        self.population = list(range(1, len(sampling_weights) + 1))
        self.sampling_weights = sampling_weights
        self.candidates = []
        self.i = 0

    def draw(self):
        if self.i == len(self.candidates):
            # Cache 10000 random choices for speed
            self.candidates = random.choices(
                self.population, self.sampling_weights, k=10000)
            self.i = 0
        self.i += 1
        return self.candidates[self.i - 1]

def get_negatives(all_contexts, vocab, counter, K):
    """Return noise words in negative sampling."""
    # Sampling probability proportional to frequency^0.75
    sampling_weights = [counter[vocab.to_tokens(i)]**0.75
                        for i in range(1, len(vocab))]
    
    all_negatives, generator = [], RandomGenerator(sampling_weights)
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            neg = generator.draw()
            # Noise word cannot be in the context
            if neg not in contexts:
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives


Below code block handles the **Data Loading and Batching** mechanics. Once we have our raw training examples (Center, Context, Negatives), we need to feed them into the neural network. However, neural networks require inputs to be structured as fixed-size Matrices (Tensors), but our text data is naturally irregular (variable lengths). This block bridges that gap.

#### **1. Purpose and Function**

The code performs three crucial operations to prepare data for the GPU:

1.  **Padding (`batchify`):** Since context windows vary in size (due to the random window effect), we pad shorter sequences with zeros so they can be stacked into a single rectangular matrix.
2.  **Mask Generation:** It creates a "mask" matrix that tells the model which data points are real and which are just padding (to be ignored).
3.  **Label Construction:** It creates the target labels for the binary classification task: `1` for context words (Positive) and `0` for noise words (Negative).

-----

#### **2. Detailed Theoretical Breakdown**

**A. The Ragged Tensor Problem**
In the previous step, we defined a random window size.

  * Example 1: Center "apple", Context ["fruit", "red"] (Length 2)
  * Example 2: Center "run", Context ["fast"] (Length 1)

If we try to stack these into a matrix, the dimensions don't match. We cannot create a Tensor of shape `(2, variable)`.

**B. Padding and Masking**
To fix this, we find the maximum length in the batch (say, 5) and fill the empty spots with a dummy value (usually 0).

  * Example 1 becomes: `["fruit", "red", 0, 0, 0]`
  * Example 2 becomes: `["fast", 0, 0, 0, 0]`

However, we must ensure the model **does not learn** from these zeros. If the model tries to predict that "run" is related to the padding token "0", it will ruin the embeddings.

  * **The Mask:** We generate a binary tensor where `1` indicates real data and `0` indicates padding. The loss function uses this mask to mathematically cancel out any error coming from the padding positions.

**C. Supervised Labels for Skip-Gram**
The Skip-Gram with Negative Sampling objective is to distinguish true context words from noise.

  * **Positives (Context):** Assigned Label **1**.
  * **Negatives (Noise):** Assigned Label **0**.
  * **Padding:** Assigned Label **0** (but masked out, so it affects nothing).

-----

#### **3. Key Code Lines & Their Roles**

**1. Dynamic Batch Sizing**

```python
max_len = max(len(c) + len(n) for _, c, n in data)
```

  * **Role:** This calculates the size of the tensor for *this specific batch*. Instead of padding everything to a global maximum (e.g., 512), which wastes memory, we only pad to the longest sequence in the current set of 512 examples. This is a standard optimization in NLP called **Dynamic Padding**.

**2. Concatenation and Padding**

```python
contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
```

  * **Role:** This creates the input vector for the "Context" side of the model. Notice how **Positives** (`context`) and **Negatives** (`negative`) are merged into a single list. This allows us to compute the scores for both types of words in a single matrix multiplication step, drastically improving GPU efficiency.

**3. Label Construction**

```python
labels += [[1] * len(context) + [0] * (len(negative) + (max_len - cur_len))]
```

  * **Role:** This builds the "Ground Truth".
      * `[1] * len(context)`: The model *should* predict these words are related.
      * `[0] * len(negative)`: The model *should* predict these words are NOT related.
      * The padding is also labeled 0, but the mask will ensure this is ignored.

**4. The Collate Function**

```python
collate_fn=batchify
```

  * **Role:** Passed to the PyTorch `DataLoader`. This function overrides the default behavior (which assumes data is already uniform) and applies our custom padding logic every time the data loader fetches a new batch.

In [4]:
#  Data Loading and Batching
# We define the `PTBDataset` and a custom `batchify` function. This collate 
# function pads short contexts with zeros so that we can process data in 
# mini-batches.


def batchify(data):
    """Return a minibatch of examples for skip-gram with negative sampling."""
    max_len = max(len(c) + len(n) for _, c, n in data)
    centers, contexts_negatives, masks, labels = [], [], [], []
    
    for center, context, negative in data:
        cur_len = len(context) + len(negative)
        centers += [center]
        # Concatenate context and noise words, then pad
        contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
        # Mask: 1 for valid tokens, 0 for padding
        masks += [[1] * cur_len + [0] * (max_len - cur_len)]
        # Labels: 1 for context (positive), 0 for noise (negative) and padding
        labels += [[1] * len(context) + [0] * (len(negative) + (max_len - cur_len))]
        
    return (torch.tensor(centers).reshape((-1, 1)), torch.tensor(contexts_negatives),
            torch.tensor(masks), torch.tensor(labels))

class PTBDataset(data.Dataset):
    def __init__(self, centers, contexts, negatives):
        assert len(centers) == len(contexts) == len(negatives)
        self.centers = centers
        self.contexts = contexts
        self.negatives = negatives

    def __getitem__(self, index):
        return (self.centers[index], self.contexts[index], self.negatives[index])

    def __len__(self):
        return len(self.centers)

def load_data_ptb(batch_size, max_window_size, num_noise_words):
    """Download the PTB dataset and then load it into memory."""
    sentences = read_ptb()
    vocab = Vocab(sentences, min_freq=10)
    subsampled, counter = subsample(sentences, vocab)
    corpus = [vocab[line] for line in subsampled]
    
    all_centers, all_contexts = get_centers_and_contexts(corpus, max_window_size)
    all_negatives = get_negatives(all_contexts, vocab, counter, num_noise_words)
    
    dataset = PTBDataset(all_centers, all_contexts, all_negatives)
    data_iter = data.DataLoader(dataset, batch_size, shuffle=True,
                                collate_fn=batchify)
    return data_iter, vocab



Below code block defines the **Neural Network Architecture** and the **Optimization Objective** (Loss Function). This is the "brain" of the Word2Vec project. It translates the mathematical formulas of Skip-Gram and Negative Sampling into executable PyTorch modules.

#### **1. Purpose and Function**

1.  **`SkipGramModel`:** Defines the learnable parameters. It creates two distinct vector spaces (Center and Context). During the forward pass, it calculates the similarity (dot product) between a center word and a list of target words (both positive context and negative noise).
2.  **`SigmoidBCELoss`:** Defines how "wrong" the model is. It treats the problem as **Binary Classification**. It forces the model to predict `1` for context words and `0` for noise words, while mathematically ignoring any padding added during batching.

-----

#### **2. Detailed Theoretical Breakdown**

**A. The Two-Embedding Paradigm**
As discussed in the theory section, Word2Vec learns two vectors for every word $i$:

1.  $\mathbf{v}_i$ (when $i$ is a center word).
2.  $\mathbf{u}_i$ (when $i$ is a context word).

The code implements this using two separate `nn.Embedding` layers. An embedding layer is essentially a lookup table that maps an integer index to a dense vector.

  * The goal of training is to maximize the dot product $\mathbf{u}_o^\top \mathbf{v}_c$ for real context words and minimize it for noise words.

**B. Batch Matrix Multiplication (BMM)**
In a single batch, we have $B$ examples. For each example, we have 1 center word and $L$ targets (a mix of context and noise).

  * Center Vector shape: $(B, 1, D)$ where $D$ is embedding dimension.
  * Context Vectors shape: $(B, L, D)$.
    To compute the dot product of the center against *all* $L$ targets simultaneously for *all* $B$ examples, we use `torch.bmm` (Batch Matrix Multiplication).

**C. Binary Cross Entropy with Logits**
The Negative Sampling objective function is:
$$J = - \left( \log \sigma(\mathbf{u}_o^\top \mathbf{v}_c) + \sum \log \sigma(-\mathbf{u}_k^\top \mathbf{v}_c) \right)$$
This is mathematically identical to **Binary Cross Entropy (BCE)** loss, where the label $y=1$ for context and $y=0$ for noise.

  * **"With Logits":** The `forward` method returns raw dot products (logits). We do not apply the Sigmoid function explicitly in the model. Instead, we use `binary_cross_entropy_with_logits`, which applies the Sigmoid internally. This is numerically more stable than applying Sigmoid then Log.

-----

#### **3. Key Code Lines & Their Roles**

**1. Defining the Learnable Parameters**

```python
self.center_embeddings = nn.Embedding(vocab_size, embed_size)
self.context_embeddings = nn.Embedding(vocab_size, embed_size)
```

  * **Role:** These lines allocate the memory for the vectors. If vocab size is 10,000 and dimension is 100, each line creates a $10,000 \times 100$ matrix of float weights. These are the $\theta$ parameters we will optimize.

**2. The Geometric Operation (Dot Product)**

```python
pred = torch.bmm(v, u.permute(0, 2, 1))
```

  * **Role:** This calculates the similarity scores.
      * `v` has shape $(B, 1, D)$.
      * `u` has shape $(B, L, D)$.
      * `u.permute(0, 2, 1)` creates the transpose $(B, D, L)$.
      * `bmm` performs $(B, 1, D) \times (B, D, L) \rightarrow (B, 1, L)$.
      * The result is a score for every context/noise word relative to the center word.

**3. Masked Loss Calculation**

```python
out = nn.functional.binary_cross_entropy_with_logits(inputs, target, weight=mask, ...)
```

  * **Role:** This calculates the loss but solves the **Padding Problem**.
      * Recall that we padded short sequences with zeros. The model produces predictions for these zeros.
      * The `mask` tensor (containing 1s for real data and 0s for padding) is passed to the `weight` argument.
      * This multiplies the loss at padding positions by 0, effectively erasing them. Without this, the model would try to learn relationships between words and the padding token, destroying performance.

In [ ]:
#  Skip-Gram Model and Loss
# Here we implement:
# 1.  **SkipGramModel**: A simple neural network that looks up embeddings for the center word and the context/noise words, then computes their dot product.
# 2.  **SigmoidBCELoss**: Binary Cross-Entropy Loss with masking (to ignore padding).


class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(SkipGramModel, self).__init__()
        # Center word embeddings (v)
        self.center_embeddings = nn.Embedding(vocab_size, embed_size)
        # Context word embeddings (u)
        self.context_embeddings = nn.Embedding(vocab_size, embed_size)

    def forward(self, center, contexts_and_negatives):
        # center shape: (batch_size, 1)
        # contexts_and_negatives shape: (batch_size, max_len)
        v = self.center_embeddings(center)
        u = self.context_embeddings(contexts_and_negatives)
        # Compute dot product between v and u
        pred = torch.bmm(v, u.permute(0, 2, 1))
        return pred

class SigmoidBCELoss(nn.Module):
    """Binary Cross-Entropy Loss with Masking."""
    def __init__(self):
        super(SigmoidBCELoss, self).__init__()

    def forward(self, inputs, target, mask=None):
        out = nn.functional.binary_cross_entropy_with_logits(
            inputs, target, weight=mask, reduction="none")
        return out.mean(dim=1)



Belwo code blck implements the **Training Loop**. This is where the actual learnin occurs. It takes the model (initialized with random numbers) and the data (prepared in previous steps), and iteratively adjusts the model's weights to minimize the prediction error.

#### **1. Purpose and Function**

1.  **Initialization:** It sets the model's initial state using **Xavier Initialization** to ensure training stability.
2.  **Optimization:** It uses **Stochastic Gradient Descent (via Adam)** to update the embedding vectors $\mathbf{v}$ and $\mathbf{u}$ based on the calculated gradients.
3.  **Loss Normalization:** It correctly scales the loss to account for the variable-length sequences created by the padding process, ensuring the model isn't penalized for "predicting" padding tokens.

-----

#### **2. Detailed Theoretical Breakdown**

**A. Weight Initialization (`xavier_uniform_`)**
Neural networks are sensitive to initial conditions.

  * If weights are too small, signals vanish (gradients $\rightarrow$ 0).
  * If weights are too large, signals explode (gradients $\rightarrow \infty$).
  * **Xavier (Glorot) Initialization** draws weights from a distribution designed to keep the variance of activations and gradients roughly constant across layers. For embeddings, this ensures that the dot products $\mathbf{u}^\top \mathbf{v}$ start in a reasonable range where the Sigmoid function has a non-zero gradient.

**B. The Optimization Algorithm (Adam)**
The code uses `torch.optim.Adam`.

  * In Word2Vec, some words are very frequent ("the") and some are rare ("abacus").
  * Standard SGD applies the same learning rate to all parameters.
  * **Adam (Adaptive Moment Estimation)** adapts the learning rate for each parameter individually. It effectively allows rare words to take larger learning steps than frequent words, which speeds up convergence significantly for sparse data like text.

**C. Backpropagation and Computational Graphs**
The line `l.sum().backward()` triggers the chain rule of calculus.

1.  PyTorch traverses the graph from the Loss back to the Embeddings.
2.  It computes $\frac{\partial J}{\partial \mathbf{v}_c}$ and $\frac{\partial J}{\partial \mathbf{u}_o}$.
3.  `optimizer.step()` subtracts these gradients from the current weights: $\theta \leftarrow \theta - \eta \cdot \nabla \theta$.

-----

#### **3. Key Code Lines & Their Roles**

**1. Xavier Initialization**

```python
nn.init.xavier_uniform_(m.weight)
```

  * **Role:** Replaces the default PyTorch initialization (which is often just $\mathcal{N}(0, 1)$ or uniform) with Xavier initialization. This is a critical "best practice" to prevent the model from getting stuck immediately at the start of training.

**2. Loss Renormalization**

```python
l = (loss_fn(...) / mask.sum(axis=1) * mask.shape[1])
```

  * **Role:** This is a subtle but crucial mathematical correction.
      * The `SigmoidBCELoss` calculates the mean loss over the *entire* row (including padding).
      * Example: A row has 2 real words and 3 pads. `loss_fn` divides the total error by 5.
      * However, we only want the average over the **2 real words**.
      * **Math:** We multiply by 5 (`mask.shape[1]`) to get the total sum back, then divide by 2 (`mask.sum(axis=1)`) to get the correct average. Without this, the gradients would be diluted by the zeros from the padding.

**3. Gradient Reset**

```python
optimizer.zero_grad()
```

  * **Role:** PyTorch accumulates gradients by default (for RNNs etc.). In this training loop, we must clear the gradients from the previous batch before calculating the new ones; otherwise, the gradients would add up indefinitely, leading to a massive step size that causes the model to diverge (Loss $\rightarrow$ NaN).

**4. GPU Transfer**

```python
center, context_negative, mask, label = [data.to(device) for data in batch]
```

  * **Role:** Moving the data tensors to the GPU memory (`cuda:0`). If the Model is on the GPU but the Data is on the CPU, PyTorch will throw a runtime error. Both must reside on the same physical device to perform the matrix operations.

In [6]:
#  Training Word2Vec
# The training loop iterates over the dataset, computes the loss, and updates the embedding weights.


def train_word2vec(net, data_iter, lr, num_epochs, device):
    def init_weights(m):
        if type(m) == nn.Embedding:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss_fn = SigmoidBCELoss()
    
    print(f"Training on {device}...")
    for epoch in range(num_epochs):
        timer = Timer()
        metric = Accumulator(2) # Sum of loss, number of tokens
        
        for i, batch in enumerate(data_iter):
            optimizer.zero_grad()
            center, context_negative, mask, label = [
                data.to(device) for data in batch]
            
            pred = net(center, context_negative)
            l = (loss_fn(pred.reshape(label.shape).float(), label.float(), mask)
                 / mask.sum(axis=1) * mask.shape[1])
            
            l.sum().backward()
            optimizer.step()
            metric.add(l.sum(), l.numel())
            
        print(f'Epoch {epoch + 1}, Loss {metric[0] / metric[1]:.3f}, '
              f'{metric[1] / timer.stop():.1f} tokens/sec')

# Run Training 
if __name__ == '__main__':
    print("--- Starting Word2Vec Training ---")
    batch_size, max_window_size, num_noise_words = 512, 5, 5
    device = try_gpu()
    data_iter, vocab = load_data_ptb(batch_size, max_window_size, num_noise_words)

    embed_size = 100
    net = SkipGramModel(len(vocab), embed_size)
    train_word2vec(net, data_iter, lr=0.002, num_epochs=5, device=device)




--- Starting Word2Vec Training ---
Extracting ./data\ptb.zip...
Training on cpu...
Epoch 1, Loss 0.481, 13192.2 tokens/sec
Epoch 2, Loss 0.427, 11001.8 tokens/sec
Epoch 3, Loss 0.404, 11411.5 tokens/sec
Epoch 4, Loss 0.380, 14056.0 tokens/sec
Epoch 5, Loss 0.360, 14425.5 tokens/sec


Below code block implements **Byte Pair Encoding (BPE)**, a subword tokenization algorithm. In the previous blocks (Word2Vec), we treated words as atomic units. If a word wasn't in the vocabulary, it became `<unk>` (Unknown). BPE solves this by breaking words down into smaller, meaningful chunks (subwords), allowing the model to handle rare words and even words it has never seen before by constructing them from known parts.

#### **1. Purpose and Function**

The code performs three distinct steps of the BPE algorithm:

1.  **Statistics Collection (`get_max_freq_pair`):** It analyzes the corpus to find which pair of adjacent symbols (e.g., 'e' and 'r') appears most frequently.
2.  **Vocabulary Update (`merge_symbols`):** It permanently merges the most frequent pair into a new, single symbol (e.g., 'er'). This iteratively builds a vocabulary of longer, meaningful subwords.
3.  **Inference / Segmentation (`segment_BPE`):** It applies the learned merge rules to tokenize new, unseen words (e.g., splitting "fatter" into "fat" + "ter" based on known subwords).

-----

#### **2. Detailed Theoretical Breakdown**

**A. The Out-Of-Vocabulary (OOV) Problem**
Traditional models (like the Skip-Gram we just built) have a fixed vocabulary.

  * **Training:** "fast", "faster", "fastest".
  * **Test:** "fasting".
  * **Result:** The model treats "fasting" as `<unk>`, losing all semantic information, even though it knows "fast".

**B. Byte Pair Encoding (BPE) Algorithm**
BPE was originally a data compression algorithm. In NLP, it works by iteratively merging frequent characters.

1.  **Initialization:** Start with a vocabulary of all individual characters. Every word is a sequence of characters: `("f", "a", "s", "t", "e", "r")`.
2.  **Counting:** Count how often every adjacent pair of symbols occurs.
      * ('f', 'a'): 5 times
      * ('e', 'r'): 20 times
3.  **Merging:** Merge the most frequent pair. 'e' and 'r' become the new symbol 'er'.
      * Word becomes: `("f", "a", "s", "t", "er")`.
4.  **Iteration:** Repeat this process $N$ times. Eventually, common words become single symbols ("fast"), while rare words remain split ("fast", "ing").

**C. Morphology Learning**
Crucially, BPE automatically learns morphological structures without explicit linguistic rules.

  * It learns that "ing", "ed", "er", and "est" are common suffixes because they appear frequently at the ends of many different words.
  * This allows the model to understand "un-friend-li-ness" even if it has never seen the full word "unfriendliness" before.

-----

#### **3. Key Code Lines & Their Roles**

**1. Finding the Best Merge Candidate**

```python
pairs[symbols[i], symbols[i + 1]] += freq
```

  * **Role:** This loop scans through every word in the dictionary. It looks at every adjacent pair of symbols (e.g., 't', 'a') and adds the word's frequency to that pair's count. This identifies the global "most popular" combination.

**2. Performing the Merge**

```python
new_token = token.replace(' '.join(max_freq_pair), ''.join(max_freq_pair))
```

  * **Role:** This performs the actual merge operation in the dictionary.
      * If `max_freq_pair` is `('t', 'a')`.
      * `' '.join(...)` creates the string `"t a"`.
      * `''.join(...)` creates the string `"ta"`.
      * The `replace` function updates the word structure: `"t a l l"` becomes `"ta l l"`.

**3. Segmentation (Inference)**

```python
while start < len(token) and start < end:
    if token[start: end] in symbols:
        cur_output.append(token[start: end])
```

  * **Role:** This is a **Greedy Matching** algorithm used when processing new text.
    1.  It tries to match the longest possible substring of the word against the known vocabulary (`symbols`).
    2.  If it finds a match (e.g., "tall"), it adds it to the output.
    3.  If not, it shrinks the window (`end -= 1`) and tries again.
    4.  This ensures that "tallest" is tokenized as `["tall", "est"]` rather than `["t", "a", "l", "l", "e", "s", "t"]`, preserving meaning.

In [7]:
#  Subword Embedding (Byte Pair Encoding)
# As discussed in the book (Section 15.6), Byte Pair Encoding (BPE) allows us 
# to build a vocabulary of subword units. This helps in handling rare or 
# out-of-vocabulary words.


def get_max_freq_pair(token_freqs):
    """Get the most frequent pair of consecutive symbols."""
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # Key is tuple of two consecutive symbols
            pairs[symbols[i], symbols[i + 1]] += freq
    return max(pairs, key=pairs.get) # Key with max value

def merge_symbols(max_freq_pair, token_freqs, symbols):
    """Merge the most frequent pair into a new symbol."""
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

def segment_BPE(tokens, symbols):
    """Segment new tokens using the learned BPE symbols."""
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
                if start >= end:
                    cur_output.append('<unk>')
                    break
        outputs.append(' '.join(cur_output))
    return outputs

# BPE Demo 
if __name__ == '__main__':
    print("\n--- Starting BPE Demo ---")
    symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
               'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
               '_', '[UNK]']
    raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
    token_freqs = {}
    for token, freq in raw_token_freqs.items():
        token_freqs[' '.join(list(token))] = raw_token_freqs[token]

    num_merges = 10
    for i in range(num_merges):
        max_freq_pair = get_max_freq_pair(token_freqs)
        token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
        print(f'Merge #{i + 1}:', max_freq_pair)



--- Starting BPE Demo ---
Merge #1: ('t', 'a')
Merge #2: ('ta', 'l')
Merge #3: ('tal', 'l')
Merge #4: ('f', 'a')
Merge #5: ('fa', 's')
Merge #6: ('fas', 't')
Merge #7: ('e', 'r')
Merge #8: ('er', '_')
Merge #9: ('tall', '_')
Merge #10: ('fast', '_')


The final code block implements **Word Similarity** and **Word Analogy** tasks. While the previous blocks focused on *training* a model from scratch, this block focuses on *evaluating* and *using* embeddings. Because the small Penn Tree Bank dataset used earlier is too small to learn complex semantic relationships, this block loads **Pretrained GloVe Embeddings** (trained on billions of words) to demonstrate the true power of distributed representations.

#### **1. Purpose and Function**

The code performs three main functions:

1.  **Loading Pretrained Vectors (`TokenEmbedding`):** It parses a large text file containing pre-computed vectors (GloVe) and loads them into memory as a lookup table.
2.  **Semantic Search (`get_similar_tokens`):** It finds words that are semantically closest to a query word (e.g., "chip" $\rightarrow$ "microprocessor").
3.  **Analogical Reasoning (`get_analogy`):** It solves algebraic puzzles of the form "A is to B as C is to ?" using vector arithmetic.

-----

#### **2. Detailed Theoretical Breakdown**

**A. Pretrained Embeddings (Transfer Learning)**
Training high-quality embeddings requires massive datasets (Wikipedia, Common Crawl) and significant compute power.

  * **GloVe (Global Vectors):** A popular set of pretrained vectors. The filename `glove.6B.50d` indicates it was trained on **6 Billion** tokens and each word is represented by a **50-dimensional** vector.
  * By loading these, we transfer the knowledge learned from the massive corpus to our local application.

**B. Cosine Similarity (`knn`)**
In the vector space, "similarity" is defined by the angle between two vectors, not the Euclidean distance.
$$\text{similarity}(\mathbf{x}, \mathbf{y}) = \cos(\theta) = \frac{\mathbf{x} \cdot \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|}$$

  * If the vectors point in the exact same direction, $\cos(0) = 1$.
  * If they are orthogonal (unrelated), $\cos(90^\circ) = 0$.
  * If they point in opposite directions, $\cos(180^\circ) = -1$.

**C. Word Analogy (Vector Arithmetic)**
One of the most famous properties of word embeddings is that semantic relationships are encoded as linear offsets in the vector space.

  * The relationship "Man $\to$ Woman" is captured by the difference vector: $\mathbf{v}_{woman} - \mathbf{v}_{man}$.
  * To find the analogue for "Son", we add this difference vector to the "Son" vector:
    $$\mathbf{v}_{target} \approx \mathbf{v}_{son} + (\mathbf{v}_{woman} - \mathbf{v}_{man})$$
  * We then search the vocabulary for the word vector closest to this result. Ideally, it should be "Daughter".

-----

#### **3. Key Code Lines & Their Roles**

**1. Efficient Cosine Calculation**

```python
cos = torch.mv(W, x.reshape(-1,)) / (
    torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
    torch.sqrt((x * x).sum()))
```

  * **Role:** This calculates the cosine similarity between the query vector `x` and **every single word** in the vocabulary `W` simultaneously.
      * `torch.mv(W, x)`: Matrix-Vector multiplication. Computes the dot product $\mathbf{w}_i \cdot \mathbf{x}$ for all $i$.
      * `torch.sum(W * W, axis=1)`: Computes $\|\mathbf{w}_i\|^2$ (squared magnitude) for all vectors.
      * `1e-9`: A tiny epsilon added to prevent division by zero errors.

**2. The Analogy Arithmetic**

```python
x = vecs[1] - vecs[0] + vecs[2]
```

  * **Role:** This single line implements the algebraic reasoning logic.
      * `vecs[0]`: Vector for word 'a' (e.g., "man").
      * `vecs[1]`: Vector for word 'b' (e.g., "woman").
      * `vecs[2]`: Vector for word 'c' (e.g., "son").
      * It computes the hypothetical position of the answer vector in the 50-dimensional space.

**3. Parsing the GloVe File**

```python
elems = line.rstrip().split(' ')
token, elems = elems[0], [float(elem) for elem in elems[1:]]
```

  * **Role:** GloVe files are simple text files where each line looks like: `apple 0.45 -0.12 0.98 ...`.
      * This code separates the string token ("apple") from its numerical components.
      * It converts the string of numbers into a list of floats, which eventually becomes a Tensor row in the embedding matrix.

In [ ]:
#  Word Similarity and Analogy
# Implement the similarity and analogy tasks are just for seeing how above models and training actually work. We download 
# pretrained GloVe vectors for this, as the small PTB dataset used above is too small to learn high-quality semantic relationships.


class TokenEmbedding:
    """Token Embedding Loader (GloVe)."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        # Download GloVe 50d
        data_dir = download_extract(embedding_name)
        
        # We specifically look for vec.txt inside the extracted folder
        # Note: The zip might extract to a folder or flat files.
        # For safety, we search for the specific txt file.
        file_path = os.path.join(data_dir, 'vec.txt')
        
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
                    
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        return self.idx_to_vec[torch.tensor(indices)]

def knn(W, x, k):
    """Find k-nearest neighbors using cosine similarity."""
    # Add 1e-9 for numerical stability
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

def get_similar_tokens(query_token, k, embed):
    """Print similar tokens."""
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

def get_analogy(token_a, token_b, token_c, embed):
    """Compute analogy: a is to b as c is to ?"""
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]

# Run Analogy Demo (Downloads GloVe)
if __name__ == '__main__':
    print("\n--- Starting Analogy/Similarity Demo ---")
    # Note: This downloads ~160MB. 
    glove_embedding = TokenEmbedding('glove.6b.50d')

    print("\nSimilar to 'chip':")
    get_similar_tokens('chip', 3, glove_embedding)

    print("\nAnalogy: man -> woman :: son -> ?")
    print(get_analogy('man', 'woman', 'son', glove_embedding))


--- Starting Analogy/Similarity Demo ---

Similar to 'chip':
cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics

Analogy: man -> woman :: son -> ?
daughter
